In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window

# https://mvnrepository.com/artifact/mysql/mysql-connector-java
# download jar spark agent https://github.com/open-metadata/openmetadata-spark-agent/releases

# https://docs.open-metadata.org/v1.3.x/deployment/security/enable-jwt-tokens#generate-token
jwtToken = 'xxxxxxxxxxxx'

spark = (SparkSession.builder
         .appName('sample_spark')
         .config('spark.extraListeners', 'org.openmetadata.spark.agent.OpenMetadataSparkListener')
         .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.30')
         .config('spark.jars', '/home/jovyan/openmetadata-spark-agent-1.0-beta.jar')
         .config("spark.openmetadata.transport.hostPort", 'http://openmetadata.openmetadata:8585')
         .config('spark.openmetadata.transport.type', 'openmetadata')
         .config('spark.openmetadata.transport.jwtToken', jwtToken)
         .config('spark.openmetadata.transport.pipelineServiceName', 'my_pipeline_service')
         .config('spark.openmetadata.transport.pipelineName', 'my_pipeline_name')
       #  .config('spark.openmetadata.transport.pipelineSourceUrl', '')
         .config('spark.openmetadata.transport.pipelineDescription', 'My ETL Pipeline')
       #  .config('spark.openmetadata.transport.databaseServiceNames', 'random,local_mysql')
         .config('spark.openmetadata.transport.timeout', '30')
         .getOrCreate()
        )

In [2]:
df_produtos = spark.read.format('parquet').load('tables/produtos')

df_produtos.show()

+----------+------------+
|produto_id|produto_nome|
+----------+------------+
|         1|       Arroz|
|         2|      Feijão|
|         4|        Óleo|
+----------+------------+



In [3]:
df_vendas = spark.read.format('parquet').load('tables/vendas')

df_vendas.show()

+----------+----------+-----+
|      data|produto_id|valor|
+----------+----------+-----+
|2022-01-01|         1|100.0|
|2022-01-02|         2| 50.0|
|2022-01-03|         3| 25.0|
+----------+----------+-----+



In [4]:
df_final = df_vendas.join(df_produtos, "produto_id", "left")

df_final.show()

+----------+----------+-----+------------+
|produto_id|      data|valor|produto_nome|
+----------+----------+-----+------------+
|         1|2022-01-01|100.0|       Arroz|
|         2|2022-01-02| 50.0|      Feijão|
|         3|2022-01-03| 25.0|        null|
+----------+----------+-----+------------+



In [5]:
df_final.write.mode('overwrite').format("parquet").save("tables/final")